<a href="https://colab.research.google.com/github/BeArnab96/MCP_Prediction/blob/master/Plant_algo_arnab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import zipfile, io, os

def upload_dir_file(case_f):
    # author: yasser mustafa, 21 March 2018  
    # case_f = 0 for uploading one File or Package(.py) and case_f = 1 for uploading one Zipped Directory
    uploaded = files.upload()    # to upload a Full Directory, please Zip it first (use WinZip)
    for fn in uploaded.keys():
        name = fn  #.encode('utf-8')
        #print('\nfile after encode', name)
        #name = io.BytesIO(uploaded[name])
    if case_f == 0:    # case of uploading 'One File only'
        print('\n file name: ', name)
        return name
    else:   # case of uploading a directory and its subdirectories and files
        zfile = zipfile.ZipFile(name, 'r')   # unzip the directory 
        zfile.extractall()
        for d in zfile.namelist():   # d = directory
            print('\n main directory name: ', d)
            return d
print('Done!')

file_name = upload_dir_file(0)

Done!


Saving Test4.csv to Test4 (1).csv
Saving time_series.py to time_series (1).py
Saving Train4.csv to Train4 (1).csv
('\n file name: ', u'Test4.csv')


In [0]:
import numpy as np
from time_series import *

Using TensorFlow backend.


In [0]:
training_file='Train4.csv'
test_file='Test4.csv'

In [0]:
npop=4
max_seeds=8
max_epochs=3
ndim=3
alpha=0.9
max_lim=np.array([120,50,5])
min_lim=np.array([20,10,2])
pop=np.zeros((npop,ndim))
fit=np.zeros(npop)
max_fit=[np.empty([1,ndim]),0]

In [0]:
params={'timestep':20,
        'dropout_prob':0.1,
        'num_units': 50,
        'num_hidden_layers':4,
        'mode': 'training'}

In [0]:
rnn=time_series_analysis(training_file,test_file)

In [0]:
pop=min_lim+np.random.rand(npop,ndim)*(max_lim-min_lim)

for i in range(npop):
  
    timestep,num_units,num_hidden_layers=pop[i,:]
    params['timestep']=int(round(timestep))
    params['num_hidden_layers']=int(round(num_hidden_layers))
    params['num_units']=int(round(num_units))
    
    fit[i]=np.divide(1,1+rnn.get_validation_error(params,1))

Epoch 1/1
22/22 [==============================] - 4s 203ms/step - loss: 0.0297
Epoch 1/1
112/112 [==============================] - 6s 51ms/step - loss: 0.0236
Epoch 1/1
97/97 [==============================] - 6s 60ms/step - loss: 0.0339
Epoch 1/1
106/106 [==============================] - 7s 64ms/step - loss: 0.0247


In [0]:
max_fit[0]=pop[np.argmax(fit),:]
max_fit[1]=max(fit)

In [0]:
numSeeds=(max_seeds*(fit/sum(fit)))

In [0]:
for g in range(max_epochs):
  
  for n in range(npop):
    
    sigma=(np.log(g+1)/(g+1))*(np.abs(max_fit[0]-pop[n,:]))
    
    bestChild_fit=0
    bestChild=[]
    
    for s in range(int(numSeeds[n])):
      
      
      if np.random.rand()>0.5:
          
        mean=max_fit[0]
        newplant=np.random.normal(mean,sigma)+np.random.randn()*max_fit[0]-np.random.randn()*pop[n,:]
        
      else:
        
        mean=pop[n,:]
        newplant=np.random.normal(mean,sigma)
        
      for d in range(ndim):
        
        newplant[d]=min(newplant[d],max_lim[d])
        newplant[d]=max(newplant[d],min_lim[d])
      
      timestep,num_units,num_hidden_layers=newplant
      params['timestep']=int(round(timestep))
      params['num_hidden_layers']=int(round(num_hidden_layers))
      params['num_units']=int(round(num_units))
      
      child_fit=np.divide(1,1+rnn.get_validation_error(params,1))
                           
      if (bestChild_fit<child_fit) :
                       bestChild_fit=child_fit
                       bestChild=newplant
                                                   
    if (fit[n]<bestChild_fit) :
                       pop[n,:]=bestChild
                       fit[n]=bestChild_fit
                              
      
                           
  max_fit[0]=pop[np.argmax(fit),:]
  max_fit[1]=max(fit)
  max_seeds=int(round(max_seeds*alpha))
  numSeeds=np.floor(max_seeds*(fit/sum(fit)))

Epoch 1/1
22/22 [==============================] - 5s 249ms/step - loss: 0.0363
Epoch 1/1
120/120 [==============================] - 7s 59ms/step - loss: 0.0396
Epoch 1/1
112/112 [==============================] - 7s 64ms/step - loss: 0.0377
Epoch 1/1
97/97 [==============================] - 7s 76ms/step - loss: 0.0325
Epoch 1/1
120/120 [==============================] - 8s 67ms/step - loss: 0.0387
Epoch 1/1
32/34 [===========================>..] - ETA: 0s - loss: 0.0361

In [0]:
timestep_opt,num_units_opt,num_hidden_opt=max_fit[0]
params['timestep']=int(round(timestep))
params['num_hidden_layers']=int(round(num_hidden_layers))
params['num_units']=int(round(num_units))

print('The optimum timestep, number of units per layer and number of hidden layers are given respectively:',round(timestep),round(num_units),round(num_hidden_layers))

In [0]:
params['mode']='predict'
rnn.plot_stock_price(params,500)